In [2]:
using Interpolations, Plots

We define the mixture setting variable $m$, with $m=0$ corresponding to best economy, $m = 0.5$ to best power and $m = 1$ to full rich.

We need the function $\pi / \pi_{pow} = f(m)$, that is, the ratio between normalized power $\pi$ for a given $(m, n, \delta, \mu)$ and the normalized power $\pi_{pow}$ for $(m=0.5, n, \delta, \mu)$.

We also need $SFC / SFC_{pow} = g(m)$, that is, the ratio between the specific fuel consumption $SFC$ for a given $(m, n, \delta, \mu)$ and the specific fuel consumption $SFC_{pow}$ for $(m=0.5, n, \delta, \mu)$. 

Both of these are given by Figure 3.1. To use it, we first define the linear relationship between x axis length (as measured in the PDF, in mm) and our newly defined variable $m$. For this we have two points: $(x = 0, m = 0)$, $(x = 99.25, m = 1)$. So:

For the linear relationship between y axis length and $f$, we have two points: $(y = 84, f = 0.86)$, $(y = 110, f = 1)$. So:

For the linear relationship between y axis length and $g$, we can get one value of $g$ from Fig. 3-5. For example, for 2400 rpm and 120 hp, $SFC_{pow} = SFC(m=0.5) = 60.8 / 120 lb/h/hp$ and $SFC_{eco} = SFC(m=0) = 51.5/120 lb/h/hp$, so $g(0) = 51.5/60.8 = 0.85$. And the other value is simply $g(0.5) = SFC(m=0.5) / SFC_{pow} = 1$, which is true by definition. The resulting two points are $(y = 4.29, g = 0.85)$, $(y = 20.12, g = 1)$.

In [3]:
x2m(x) = x/99.25
y2f(y) = 0.86 + (1-0.86)*(y-84)/(110-84)
y2g(y) = 0.85 + (1 - 0.85)*(y - 4.29)/(20.12 - 4.29)

y2g (generic function with 1 method)

In [4]:
y2f(101.01)
y2g(6.45)

0.8704674668351232

In [12]:
x_data = [0, 7.08, 13.61, 27.92, 39.87, 46.82, 53.76, 63.21, 80.99, 99.33]
m_data = x_data |> x2m

y_f_data = [83.95, 97.15, 102.71, 108.03, 109.8, 110.5, 109.8, 108.9, 105.56, 100.73]
f_data = y2f.(y_f_data)
y_g_data = [6.43, 4.29, 4.76, 9.64, 15.84, 20.12, 24.53, 31.15, 45.79, 65.26]
g_data = y2g.(y_g_data)


π_ratio = LinearInterpolation(m_data, f_data, extrapolation_bc = Flat())
sfc_ratio = LinearInterpolation(m_data, g_data, extrapolation_bc = Flat())


@show m_data
@show f_data
@show g_data

@show π_ratio(0.5)
@show sfc_ratio(0.5)

m_data = [0.0, 0.07133501259445844, 0.13712846347607052, 0.2813098236775819, 0.401712846347607, 0.47173803526448366, 0.5416624685138539, 0.6368765743073048, 0.8160201511335012, 1.0008060453400505]
f_data = [0.8597307692307692, 0.9308076923076923, 0.9607461538461538, 0.9893923076923077, 0.9989230769230769, 1.0026923076923078, 0.9989230769230769, 0.9940769230769231, 0.9760923076923077, 0.9500846153846154]
g_data = [0.8702779532533165, 0.85, 0.8544535691724573, 0.9006948831332912, 0.959444093493367, 1.0, 1.0417877447883765, 1.104516740366393, 1.2432406822488944, 1.4277321541377133]
π_ratio(0.5) = 1.0011688649966748
sfc_ratio(0.5) = 1.0168897152927083


1.0168897152927083

Let's go to figure 3-5.

First, two points to map y-axis length to fuel consumption: $(y = 46.57, fc = 50), (y = 120.73, fc = 90)$.

In [13]:
y2fc(y) = 50 + (90 - 50) * (y - 46.57) / (120.73 - 46.57)

n_data = [2000, 2200, 2400, 2600, 2700] / 2700
π_data = [0.45, 1]

y_data = [ 37.93  109.14
           41.78  116.6
           45.73  121.65
           48.82  125.1
           52.65  127.43]

fc_data = y2fc.(y_data)
fc_data *= 0.454 / 3600 #convert lb/h to kg/s

fc_interp = LinearInterpolation((n_data, π_data), fc_data, extrapolation_bc = Line())

π_knots = 10 .^ range(-1, 0, length = 8)
P_knots = π_knots * 200 #hp
P_knots *= 745.7 #to W

sfc_data = Array{Float64, 2}(undef, length(n_data), length(π_knots))
for (i, n) in enumerate(n_data)
    fc_knots = fc_interp(n, π_knots)
    sfc_data[i, :] = fc_knots ./ P_knots #compute sfc for this n over P
end

# @show sfc_data #this is now in kg/s/W

# plot(π_knots, sfc_data')

sfc_interp = LinearInterpolation((n_data, π_knots), sfc_data, extrapolation_bc = Line())

n_plot = range(0.2, 1, length = 10)
π_plot = range(0.1, 1, length = 50)
# plot(π_plot, sfc_interp(n_plot, π_plot)')

@show n_data
@show π_knots
@show sfc_data

n_data = [0.7407407407407407, 0.8148148148148148, 0.8888888888888888, 0.9629629629629629, 1.0]
π_knots = [0.1, 0.13894954943731377, 0.193069772888325, 0.2682695795279726, 0.37275937203149406, 0.5179474679231212, 0.719685673001152, 1.0]
sfc_data = [1.767095633363639e-7 1.437282022814357e-7 1.1999198925412573e-7 1.0290937680706742e-7 9.061526537148842e-8 8.176736950902258e-8 7.539966562059972e-8 7.08169203621881e-8; 1.837913797604591e-7 1.4966403350624411e-7 1.251030713495361e-7 1.0742689877022986e-7 9.470561061140735e-8 8.555028178138359e-8 7.896132279080107e-8 7.421934340528672e-8; 1.9861426835304688e-7 1.6058755055359163e-7 1.3322026656206582e-7 1.1352442436441092e-7 9.934960891706715e-8 8.914819732217842e-8 8.180638755294917e-8 7.6522592248135e-8; 2.116625565770675e-7 1.700618991348063e-7 1.4012250198618226e-7 1.1857554680002592e-7 1.0306851185575137e-7 9.190832097562875e-8 8.387649149124636e-8 7.809609888334817e-8; 2.3348433634288296e-7 1.8541804435332762e-7 1.5082542265415462e-7 1.

5×8 Matrix{Float64}:
 1.7671e-7   1.43728e-7  1.19992e-7  …  8.17674e-8  7.53997e-8  7.08169e-8
 1.83791e-7  1.49664e-7  1.25103e-7     8.55503e-8  7.89613e-8  7.42193e-8
 1.98614e-7  1.60588e-7  1.3322e-7      8.91482e-8  8.18064e-8  7.65226e-8
 2.11663e-7  1.70062e-7  1.40123e-7     9.19083e-8  8.38765e-8  7.80961e-8
 2.33484e-7  1.85418e-7  1.50825e-7     9.51177e-8  8.58376e-8  7.91588e-8

In [7]:
#now we have all we need. let's run an example
n_val = 0.7
m_val = 0.1

#power output for best power mixture setting
π_pow = 0.6
#actual power output, accounting for mixture setting
@show π_actual = π_pow * π_ratio(m_val) #correct pow
#sfc if the actual power output were obtained with best power mixture setting
@show sfc_pow = sfc_interp(n_val, π_actual)
#actual sfc, accounting for mixture setting
@show sfc_actual = sfc_pow * sfc_ratio(m_val) #kg/s/W

P_rated = 200 * 745.7 #W
@show P_actual = π_actual * P_rated #W
@show fuel_flow = sfc_actual * P_actual #kg/s
@show fuel_flow_per_h = fuel_flow * 3600

π_actual = π_pow * π_ratio(m_val) = 0.5663107951466603
sfc_pow = sfc_interp(n_val, π_actual) = 7.803204291988158e-8
sfc_actual = sfc_pow * sfc_ratio(m_val) = 6.647864498600496e-8
P_actual = π_actual * P_rated = 84459.5919881729
fuel_flow = sfc_actual * P_actual = 0.005614759231444575
fuel_flow_per_h = fuel_flow * 3600 = 20.21313323320047


20.21313323320047

In [8]:
1.1835e-7 * π_actual * P_rated

0.009995792711800264